# Generate CyRSoXS files
* Load morphology data as numpy arrays
* Add relevant parameters for Euler Morphology structure
* Save morphology hdf5
* Load optical constants data for materials
* Write optical constants to correctly formatted .txt files
* Write configuration file for CyRSoXS simulation instructions 

In [ ]:
### Imports:
import numpy as np
import matplotlib.pyplot as plt
import sys
import pathlib
from skimage.filters import sobel_v, sobel_h, sobel, gaussian

# import PyHyperScattering as phs
# from PyHyperScattering.load import cyrsoxsLoader as load
# from PyHyperScattering.integrate import WPIntegrator as integ
# print(f'Using PyHyperScattering version: {phs.__version__}')

# sys.path.append('/Users/andrew/onedriveCU/research/data_analysis/repos/NRSS')
from NRSS.writer import write_materials, write_hdf5, write_config, write_slurm
from NRSS.visualizer import morphology_visualizer
from NRSS.morphology import Morphology

In [ ]:
np.sqrt(200*200*10)

In [ ]:
(500**2)/6250

In [ ]:
(632**2)/4000

In [ ]:
(200*200*10)/816

In [ ]:
(5*34)

3 nodes & 102 total cores good for 3d morphs 134x134x10 vox

In [ ]:
vpc2eff = {}  # voxels per core to seff for running 3d models

In [ ]:
vox = (134**2*10)
cores = 100

print(f'voxels = {vox}')
print(f'cores = {cores}')
print(f'voxels per core = {int(vox/cores)}')

### Run morph gen

In [ ]:
## This was only true for 40x40x10 vox, cant get to run at 200x200x10

vpc2eff[int(vox/cores)] = (65.80, 'fast! 19 seconds')

print('Voxels per Core | CPU Efficiency')
for k, v in vpc2eff.items():
    print(f'{k:>15} | {v}')

For 2D morphologies, 4 nodes * 25 cores/node was good for large 632x632 


In [ ]:
500/60

In [ ]:
### Define paths:
notebookPath = pathlib.Path.cwd()
rootPath = notebookPath.parent
morphsPath = notebookPath.joinpath('fipy_morphs/2D')
optsPath = rootPath.joinpath('nexafs_analysis/optical_constants')

### Set colormap:
cmap = plt.cm.YlGnBu_r.copy()

In [ ]:
optsPath

In [ ]:
sorted([f.name for f in optsPath.glob('*')])

In [ ]:
sorted([f.name for f in morphsPath.glob('*steps*.txt')])

In [ ]:
morph = np.loadtxt(list(morphsPath.glob('200*500steps*1.txt'))[0])
morph_bin = np.loadtxt(list(morphsPath.glob('200*500steps*y.txt'))[0])

In [ ]:
from matplotlib_inline.backend_inline import set_matplotlib_formats

In [ ]:
set_matplotlib_formats('svg')

In [ ]:
plt.imshow(morph, origin='lower', cmap=cmap, extent=(0, 100, 0, 100))
plt.xlabel('[nm]')
plt.ylabel('[nm]')
plt.colorbar(label='Material Volume Fraction')

In [ ]:
### Define volume fraction, S, theta, and psi arrays. Let mat1 be the donor and mat2 be the acceptor:
mat1_vfrac = morph
mat1_S = np.zeros(morph.shape)
mat1_theta = np.zeros(morph.shape)
mat1_psi = np.zeros(morph.shape)

mat2_vfrac = 1-morph
mat2_S = np.zeros(morph.shape)
mat2_theta = np.zeros(morph.shape)
mat2_psi = np.zeros(morph.shape)

In [ ]:
### Reshape to (Z, Y, X):
mat1_vfrac = np.reshape(mat1_vfrac, (1, morph.shape[1], morph.shape[0]))
mat1_S = np.reshape(mat1_S, (1, morph.shape[1], morph.shape[0]))
mat1_theta = np.reshape(mat1_theta, (1, morph.shape[1], morph.shape[0]))
mat1_psi = np.reshape(mat1_psi, (1, morph.shape[1], morph.shape[0]))

mat2_vfrac = np.reshape(mat2_vfrac, (1, morph.shape[1], morph.shape[0]))
mat2_S = np.reshape(mat2_S, (1, morph.shape[1], morph.shape[0]))
mat2_theta = np.reshape(mat2_theta, (1, morph.shape[1], morph.shape[0]))
mat2_psi = np.reshape(mat2_psi, (1, morph.shape[1], morph.shape[0]))


In [ ]:
write_hdf5([[mat1_vfrac, mat1_S, mat1_theta, mat1_psi], [mat2_vfrac, mat2_S, mat2_theta, mat2_psi]],
           PhysSize=1.0, fname=H5morphPath.joinpath('350x350pix_500x500size_500steps_D1_a1_epsilon1.hdf5'))

In [ ]:
morph_obj = Morphology.load_morph_hdf5(H5morphPath.joinpath('350x350pix_500x500size_500steps_D1_a1_epsilon1.hdf5'))
morph_obj.materials[1].name = 'PM6'
morph_obj.materials[1].name = 'Y6'
morphology_visualizer(morph_obj, vfrac_cmap=cmap)

In [ ]:
energies = np.round(np.arange(275,300,0.1),1)

material_dict = {'Material1': optsPath.joinpath('PM6_molecular.txt'), 
                 'Material2': optsPath.joinpath('Y6_molecular.txt')}
energy_dict = {
    'Energy': 0,
    'DeltaPara': 1,
    'BetaPara': 2,
    'DeltaPerp': 3,
    'BetaPerp': 4
}

write_materials(energies, material_dict, energy_dict, 2)

